# Putting Jedi Masters in the Training Rooms - A QAP Example

The Jedi Temple on Dantooine is building 4 training rooms  (1, 2, 3, 4) for Padawans. There is a walking distance (in feet) between all pairs of rooms ($d_{ij}$) Jedi Masters (Asnar, Bendak, Chorla, Duzek) need to be assigned to one of the 4 rooms. Make assignment so Padawans travelling between rooms don’t need to walk too far. We know how many Padawans will need to travel between each pair of Jedi Masters ($f_{ij}$).


In [1]:
using JuMP, NamedArrays, Gurobi

J = [:A,:B,:C,:D] # jedi masters
R = [ 1, 2, 3, 4] # training rooms

# create a NamedArray representing the # of padawans that move between each pair of masters
f = NamedArray([ 0 5 2 7
                 5 0 3 8
                 2 3 0 3
                 7 8 3 0 ], (J,J))
# create a NamedArray representing the distances between each pair of rooms
d = NamedArray([   0  80 150 170
                  80   0 130 100
                 150 130   0 120
                 170 100 120   0 ], (R,R))

m = Model(Gurobi.Optimizer)
set_optimizer_attribute(m, "OutputFlag", 0)

# create binary variables to determine which master is placed in each room
@variable(m, x[J,R], Bin)
# create binary variables for every combination of jedi, room, jedi, room quadruplet
# this will allow us to "linearize" the objective
@variable(m, z[J,R,J,R], Bin)

for i in J
    @constraint(m, sum( x[i,j] for j in R ) == 1)   # each Master is in exactly one room
end

for j in R
    @constraint(m, sum( x[i,j] for i in J ) == 1)   # each room has exactly one Master
end

# create a set of constraints that linearizes the product of x[i,j] and x[k,l]
for i in J
    for j in R
        for k in J
            for l in R
                @constraints(m, begin
                    x[i,j] >= z[i,j,k, l ]
                    x[k, l ] >= z[i,j,k,l]
                    x[i,j] + x[k,l] <= z[i,j,k,l] + 1
                end)
            end
        end
    end
end

# minimze the total distance the padawans must travel
# because of structure of objective, everything is "double counted." divide by 2 to scale.
@objective(m, Min, 1/2*sum( f[i,k]*d[j,l]*z[i,j,k,l] for i in J, j in R, k in J, l in R ))

optimize!(m)
# store the solution in a NamedArray for easier reading
sol = NamedArray(zeros(Int,4,4),(J,R),("Master","Room"))
for i in J
    for j in R
        sol[i,j] = value(x[i,j])
    end
end
println(sol)

Academic license - for non-commercial use only - expires 2022-06-27
4×4 Named Matrix{Int64}
Master ╲ Room │ 1  2  3  4
──────────────┼───────────
:A            │ 1  0  0  0
:B            │ 0  0  0  1
:C            │ 0  0  1  0
:D            │ 0  1  0  0
